In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.signal as si
import mezze.channel as ch
import matplotlib.pyplot as pl
from mezze.random.SchWARMA import ARMA,correlated_error
#from scipy.misc import factorial
import scipy.optimize as op
%matplotlib inline

In [ ]:
# This just checks that the fidelity numbers work regardless of the correlation time
err = correlated_error('zrot',corr_time=0,fidelity=.99)
fs = np.mean([np.real(c.chi()[0,0]) for c in err.error_stream(10000, as_channel=True)])
print(fs)

err = correlated_error('zdephasing',corr_time=5,fidelity=.96)
fs = np.mean([np.real(c.chi()[0,0]) for c in err.error_stream(10000, as_channel=True)])
print(fs)


err = correlated_error('depolarizing',corr_time=3,fidelity=.975)
fs = np.mean([np.real(c.chi()[0,0]) for c in err.error_stream(10000, as_channel=True)])
print(fs)

err = correlated_error('amp_damping',corr_time=7,fidelity=.999)
fs = np.mean([np.real(c.chi()[0,0]) for c in err.error_stream(10000, as_channel=True)])
print(fs)

While you can get e.g. PTMs by using `QuantumChannel` objects, the easiest way to interface would be to get parameters directly by setting `as_channel=False` in the `next_err` or `error_stream` methods. The below cells shows how to translate the parameter output of the error to a PTM (c.f. `quantumsim.ptm`).  Note that I use the `rgen` parameter with a constant seed so that you can see that the PTMs generated the two ways are the same.

In [ ]:
err = correlated_error('zrot',corr_time=0,fidelity=.99, rgen=np.random.RandomState(0))
param = err.next_err(as_channel=False)
print(param)
print(param, np.cos(np.arccos(np.real(param))*2), np.sin(np.arccos(np.real(param))*2))
zrot_ptm = np.diag([1,np.cos(np.arccos(np.real(param))*2),np.cos(np.arccos(np.real(param))*2),1])
zrot_ptm[1,2] = np.sin(np.arccos(np.real(param))*2)
zrot_ptm[2,1] = -np.sin(np.arccos(np.real(param))*2)
#c.f. quantumsim.ptm.rotate_z_ptm(np.arccos(np.real(param))*2)
print(zrot_ptm)
same_err = correlated_error('zrot',corr_time=0,fidelity=.99, rgen=np.random.RandomState(0))
print(same_err.next_err(as_channel=True).ptm())

In [ ]:
err = correlated_error('zdephasing',corr_time=0,fidelity=.99, rgen=np.random.RandomState(0))
dephase_param = err.next_err(as_channel=False)
print(dephase_param, 2*dephase_param -1)
#c.f. quantumsim.ptm.dephasing_ptm(1-(2*dephase_param-1),1-(2*dephase_param-1),0)
dephase_ptm = np.diag([1,2*dephase_param-1,2*dephase_param-1,1])
print(dephase_ptm)
same_err = correlated_error('zdephasing',corr_time=0,fidelity=.99, rgen=np.random.RandomState(0))
print(same_err.next_err(as_channel=True).ptm())

In [ ]:
#Depolarizing case
err = correlated_error('depolarizing',corr_time=3,fidelity=.975, rgen=np.random.RandomState(0))
depol_param = err.next_err(as_channel=False)
print(depol_param, depol_param-(1-depol_param)/3)
#c.f. quantumsim.ptm.dephasing_ptm(1-(depol_param-(1-depol_param))/3,1-(depol_param-(1-depol_param)/3),1-(depol_param-(1-depol_param)/3))
depol_ptm = np.diag([1,depol_param-(1-depol_param)/3,depol_param-(1-depol_param)/3,depol_param-(1-depol_param)/3])
print(depol_ptm)
same_err  = correlated_error('depolarizing',corr_time=3,fidelity=.975, rgen=np.random.RandomState(0))
print(same_err.next_err(as_channel=True).ptm())

In [ ]:
#Damping Case
#Depolarizing case
err = correlated_error('amp_damping',corr_time=3,fidelity=.975, rgen=np.random.RandomState(0))
gamma = err.next_err(as_channel=False)
print(gamma, np.sqrt(1-gamma), 1-gamma)
damp_ptm = np.diag([1, np.sqrt(1-gamma), np.sqrt(1-gamma), 1-gamma])
damp_ptm[3,0] = gamma
#c.f. quantumsim.ptm.amp_ph_damping_ptm(gamma,0)
print(damp_ptm)
same_err  = correlated_error('amp_damping',corr_time=3,fidelity=.975, rgen=np.random.RandomState(0))
print(same_err.next_err(as_channel=True).ptm())

For `zrot` this returns $exp(i2\theta)$, for `depolarzing` and `zdephasing` this returns the probability $\chi_{1,1}$ coefficient, i.e., the probability that the error is the identity map (no error). The errors for the other Paulis can be computed accordingly.  Thus, `zdephasing` and `zrot` can effectively be used to compute dephasing/coherent rotation errors in other dimensions.  For `amp_damping`, the decay parameter $\gamma$ is returned.

# Everything Below Here Is Check/Backup for Posterity

The below cell proves out the correlation time, defined here using a Gaussian window, such that the correlation time is apprximately the time where the autocorrelation function is equal to $\exp(-4)$, a standard convention in signal processing.

In [ ]:
#Initialize Filters
b0 = .02 #markovian
corr_times = list(range(1,21))
filter_orders = 2*np.array(corr_times)+1

bs = [np.array([1.])]+[si.gaussian(fo,std=ct/4.) for ct,fo in zip(corr_times,filter_orders)]
bs = [b/np.sqrt(np.sum(b**2))*b0 for b in bs]

# for b in bs:
#     w, h = si.freqz(b=b, a =[1])
#     pl.plot(w/(2*np.pi), 10*np.log10(abs(h)))
# pl.legend([1]+cut_offs)
# pl.show()

max_order = 1000
acv = np.zeros((len(bs),max_order))
for i,b in enumerate(bs):
    bb = np.zeros(max_order)
    bb[:len(b)] = b
    for tau in range(max_order):
        acv[i,tau] = np.sum(bb[:max_order-tau]*bb[tau:])

pl.figure()
pl.plot(acv.T/acv.T[0])
pl.legend([0]+corr_times,)
#pl.xscale('log')
pl.xlim(0,2*np.max(corr_times))
pl.plot([0,2*np.max(corr_times)],[np.exp(-4),np.exp(-4)],'--')
pl.yscale('log')
pl.ylim([.001,1])
pl.grid()
pl.ylabel('Autocorrelation')
pl.xlabel('Time')
pl.show()